# M&A Transactions Data Cleaning Pipeline

## Phase 1: data loading, cleaning, and transformation
 
This notebook prepares raw M&A transaction data for quantitative analysis by:
1. Loading and renaming columns to standardized names
2. Converting data types (dates, numeric values)
3. Converting deal values from millions to USD
4. Filtering out records with missing critical fields

In [65]:
from pathlib import Path

import polars as pl

In [66]:
# File paths
RAW_DATA_PATH = Path("../../data/raw/ma_transactions.csv")
PROCESSED_DATA_PATH = Path("../../data/processed/1.1_cleaned_data.csv")

In [67]:
# Column mapping: raw CSC names -> standardized names
COLUMN_MAPPING = {
    "All Transactions Announced Date (Including Bids and Letters of Intent)": "announced_date",
    "Total Transaction Value ($USDmm, Historical rate)": "deal_value_mm",
    "Target/Issuer": "target_name",
    "Buyers/Investors": "buyer_name",
    "Transaction Comments": "comments",
    "Transaction Status": "status",
    "CIQ Transaction ID": "transaction_id",
}

### Load raw data

In [68]:
print(f"Loading data from: {RAW_DATA_PATH}")
df_raw = pl.read_csv(RAW_DATA_PATH)
initial_count = len(df_raw)
print(f"Initial Row Count: {initial_count:,}")

Loading data from: ..\..\data\raw\ma_transactions.csv
Initial Row Count: 38,655


In [69]:
print("Raw Data Sample:")
print(df_raw.head(3))
print("Raw Column Names:")
print(df_raw.columns)

Raw Data Sample:
shape: (3, 14)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ All Trans ┆ All Trans ┆ Target/Is ┆ Exchange: ┆ … ┆ Transacti ┆ Country/R ┆ Country/R ┆ Target   │
│ actions   ┆ actions   ┆ suer      ┆ Ticker    ┆   ┆ on        ┆ egion of  ┆ egion of  ┆ Security │
│ Announced ┆ Announced ┆ ---       ┆ ---       ┆   ┆ Comments  ┆ Incorpora ┆ Incorpora ┆ Types    │
│ Dat…      ┆ Dat…      ┆ str       ┆ str       ┆   ┆ ---       ┆ tio…      ┆ tio…      ┆ ---      │
│ ---       ┆ ---       ┆           ┆           ┆   ┆ str       ┆ ---       ┆ ---       ┆ str      │
│ str       ┆ str       ┆           ┆           ┆   ┆           ┆ str       ┆ str       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 17/05/202 ┆ 17/05/202 ┆ Warner    ┆ -         ┆ … ┆ Discovery ┆ United    ┆ United    ┆ Common   │
│ 1         ┆ 1         ┆ Media,    ┆           ┆   ┆ , Inc

### Data Transformation pipeline

#### Step 1: select and rename columns

In [70]:
df_clean = df_raw.select(
    [
    pl.col(old_name).alias(new_name)
    for old_name, new_name in COLUMN_MAPPING.items()
    ]
)
print("Columns after renaming:")
print(df_clean.columns)

Columns after renaming:
['announced_date', 'deal_value_mm', 'target_name', 'buyer_name', 'comments', 'status', 'transaction_id']


#### Step 2: Parse and clean data types

In [71]:
df_clean = df_clean.with_columns(
    [
        # 1. Clean 'deal_value_mm' (remove spaces/commas) and cast to float
        pl.col("deal_value_mm")
        .str.replace_all(r"[\s,]", "")
        .cast(pl.Float64, strict=False)
        .alias("deal_value_mm_cleaned"),

        # 2. Parse 'announced_date' from DD/MM/YYYY to datetime
        pl.col("announced_date")
        .str.to_date("%d/%m/%Y", strict=True)
        .alias("announced_date"),
    ]
).drop("deal_value_mm")  # Drop original uncleaned column

print("Data types after cleaning:")
df_clean.schema

Data types after cleaning:


Schema([('announced_date', Date),
        ('target_name', String),
        ('buyer_name', String),
        ('comments', String),
        ('status', String),
        ('transaction_id', String),
        ('deal_value_mm_cleaned', Float64)])

### Step 3: convert deal value from millions to USD

In [72]:
df_clean = df_clean.with_columns(
    (pl.col("deal_value_mm_cleaned") * 1_000_000)
    .alias("deal_value_usd")
).drop("deal_value_mm_cleaned")

### Step 4: filter records with critical fields missing

In [73]:
initial_count = len(df_clean)

df_clean = df_clean.filter(
    pl.col("deal_value_usd").is_not_null() &
    pl.col("announced_date").is_not_null() &
    pl.col("target_name").is_not_null() &
    pl.col("buyer_name").is_not_null()
)

final_count = len(df_clean)
removed_count = initial_count - final_count

print(f"Rows before filtering: {initial_count:,}")
print(f"Rows after filtering:  {final_count:,}")
print(f"Rows removed:          {removed_count:,}")
print(f"Retention rate:        {final_count/max(initial_count, 1)*100:.1f}%")

Rows before filtering: 38,655
Rows after filtering:  38,655
Rows removed:          0
Retention rate:        100.0%


### Inspect cleaned data

In [74]:
print("Final schema:")
for name, dtype in df_clean.schema.items():
    print(f" - {name:<15}: {dtype}")

Final schema:
 - announced_date : Date
 - target_name    : String
 - buyer_name     : String
 - comments       : String
 - status         : String
 - transaction_id : String
 - deal_value_usd : Float64


In [77]:
print("Sample of cleaned data:")
df_clean.head(5)

Sample of cleaned data:


announced_date,target_name,buyer_name,comments,status,transaction_id,deal_value_usd
date,str,str,str,str,str,f64
2021-05-17,"""Warner Media, LLC""","""Warner Bros. Discovery, Inc. (…","""Discovery, Inc. (NasdaqGS:DISC…","""Closed""","""IQTR715171572""",1.0397e11
2019-01-02,"""Celgene Corporation""","""Bristol-Myers Squibb Company (…","""Bristol-Myers Squibb Company (…","""Closed""","""IQTR597000614""",9.9554e10
2019-03-27,"""Saudi Basic Industries Corpora…","""Saudi Arabian Oil Company (SAS…","""Saudi Arabian Oil Company sign…","""Closed""","""IQTR607827179""",9.4221e10
2019-06-09,"""Raytheon Company""","""RTX Corporation (NYSE:RTX)""","""United Technologies Corporatio…","""Closed""","""IQTR620098689""",9.3157e10
2019-06-23,"""Allergan plc""","""AbbVie Inc. (NYSE:ABBV)""","""AbbVie Inc. (NYSE:ABBV) agreed…","""Closed""","""IQTR625814909""",8.6223e10


In [78]:
# Basic statistics on deal values for quality assurance
print("Deal Value Statistics (USD):")
print(df_clean.select([
    pl.col("deal_value_usd").min().alias("min"),
    pl.col("deal_value_usd").median().alias("median"),
    pl.col("deal_value_usd").mean().alias("mean"),
    pl.col("deal_value_usd").max().alias("max"),
]))

Deal Value Statistics (USD):
shape: (1, 4)
┌─────┬────────┬──────────┬───────────┐
│ min ┆ median ┆ mean     ┆ max       │
│ --- ┆ ---    ┆ ---      ┆ ---       │
│ f64 ┆ f64    ┆ f64      ┆ f64       │
╞═════╪════════╪══════════╪═══════════╡
│ 5e7 ┆ 1.5e8  ┆ 5.9803e8 ┆ 1.0397e11 │
└─────┴────────┴──────────┴───────────┘


In [79]:
df_clean.write_csv(PROCESSED_DATA_PATH)